# ENEM 2018
For this project, I will be using the dataset used in my [Capstone project](https://github.com/davidsondefaria/Capstone) of the Udacity Data Engineering course. Such dataset is composed of Brazilian demographic and educational data. In this course, I used the knowledge acquired to carry out the ETL process for the dataset.

Now, starting the Data Science course, I will perform some analysis on this dataset. I intend to analyze the relationship between the grades obtained in ENEM, the *Exame Nacional do Ensino Médio* (in English 'High School National Exam', an application test for Universities) with the educational HDI of the Brazilian cities.

## Imports

In [1]:
import re
import os
import pandas as pd
from utils import print_pretty
from statisticalAnalysis import avgGrade
from treatData import treatCities, treatEnem

## 1. Business Understanding
In this project, I will be interested in analyzing some influences that location can have on a student who is applying for university. Here are some questions to ask.
1. Are the grades obtained in ENEM proportional to the educational HDI of the students' city?
2. How can the type of school influence the grade?
3. Is there any influence of other HDIs on the grade? (GNI or Life expectancy)
4. Is there a difference in the grade of students who live in the same cities and who have a disability compared to those who do not?
5. Does the color/race, gender or age of students influence their results or does the influence come from the place and situation in which they live? 

## 2. Data Understanding

Two datasets of different bases were used:

- [Brazil Cities](https://www.kaggle.com/crisparada/brazilian-cities): this dataset is a compilation of data on Brazilian cities taken from different websites. Although there are many interesting fields for analysis, the focus of the project is to prepare a basis for the analysis of HDI data.
- [Enem2018](http://portal.inep.gov.br/web/guest/microdados): this dataset presents all the non-sensitive data of the students who took the ENEM 2018. Through it, we can analyze the grades of each student by city, by age, by financial conditions, if they have any disabilities and other specifics.

### Source Files
The files can be found in [Google Drive](https://drive.google.com/drive/folders/1BoA9AlCZWviPwzGHz71rrIgKCUXGHr2q).

- `brazil_cities.csv`: Original dataset about cities.
- `brazil_cities_dictionary.csv`: File with subtitles for the dataset columns. 
- `enem/enem_2018.csv`: Dataset about ENEM 2018. This dataset has previously been reduced its number of columns due to its size. But keeping the original name of the columns in Portuguese.
- `enem/enem_2018_dictionary.csv` and `enem/enem_2018_dictionary.xlsx`: Both files have subtitles for ENEM columns. In Portuguese.

### Dataset Content
The content of the ENEM dataset is already very well organized by INEP, *Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira* (in English National Institute of Educational Studies and Research Anísio Teixeira - it is the federal institute linked to the Brazilian Ministry of Education and aims to promote periodic studies, research and evaluations education system).

Although the name of the columns provides a very good semantic sense, they are written in Portuguese, which can make it difficult for those who do not understand the language.

There is some missing data that can be easily corrected by assigning values without changing the semantic value of the column. This is due to the fact that the missing data refer to the test scores of students who were not present.

Some categorical data has already been worked on by INEP, with integer values that represent real values being assigned. To analyze such data, it is necessary to consult the reference file provided by the institute. However, there are still some categorical data that have not been worked on.

In [2]:
# # Comment this if you have already done the data processing process.

# # Original dataset path ENEM
# enem_2018_path = os.getcwd() + '/data/original/enem_2018.csv'
# enem_2018_df = pd.read_csv(enem_2018_path, delimiter=";")

# print(enem_2018_df.shape)
# enem_2018_df

On the other hand, the dataset of Brazilian cities is compiled from public data from multiple sources available in [Kaggle] (https://www.kaggle.com/crisparada/brazilian-cities) that aims to help students who wish improve your analytical skills. As it is a dataset composed of several sources, some data have different update dates. However, the data I want to analyze is all from the same time, which does not influence the final analysis.

There are many columns with missing data that could be dealt with by assigning values because it is information that does not exist in cities. However, these data will be taken for final analysis. In addition, there are no categorical data to be analyzed.

In [3]:
# # Comment this if you have already done the data processing process.

# # Original dataset path Brazil Cities
# brazil_cities_path = os.getcwd() + '/data/original/brazil_cities.csv'
# brazil_cities_df = pd.read_csv(brazil_cities_path, delimiter=";")

# print(brazil_cities_df.shape)
# brazil_cities_df

## 3. Prepare Data
Both datasets contain many columns that do not influence the analysis I want, so I will perform a cleaning process to keep only the columns below. I also renamed the columns.

##### Brazil Cities Columns Subtitles

|   Columns   |               Legend                  |
|-------------|---------------------------------------|
|city         |Name of Cities                         |
|state        |State of Cities                        |
|capital      |Is State Capital?                      |
|hdi_ranking  |Human Development Index Ranking        |
|hdi          |Human Development Index                |
|hdi_gni      |Human Development Index GNI per Capita |
|hdi_life     |Human Development Index Life Expectancy|
|hdi_education|Human Development Index Educational    |
|longitude    |Longitude                              |
|latitude     |Latitude                               |
|altitude     |Altitude                               |

##### Enem 2018 Columns Subtitles

|         Columns            | Legend                       |         Columns        | Legend                          |
|----------------------------|------------------------------|------------------------|---------------------------------|
|registration                |Number of Registration        |def_dyslexia            |Is dyslexic?                     |
|city_residence_code         |Code of Residence City        |def_dyscalculia         |Has dyscalculia?                 |
|city_residence              |Residence City                |def_autism              |Is autistic?                     |
|state_residence_code        |Code of Residence State       |def_monocular_vision    |Has Monocular Vision?            |
|state_residence             |Residence State               |def_other               |Has any Other Disability?        |
|age                         |Age                           |social_name             |Social Name                      |
|gender*                     |Gender                        |city_test_code          |Code of Application City         |
|matiral_status*             |Marital Status                |city_test               |Application City                 |
|color_race*                 |Color or Race                 |state_test_code         |Code of Application State        |
|nationality*                |Nationality                   |state_test              |Application State                |
|high_school_status*         |Has finished High School?     |presence_natural_science|Presence in Natural Science Test*|
|high_school_year_conclusion*|Year of High School Conclusion|presence_human_science  |Presence in Human Science Test*  |
|school_type*                |Type of High School           |presence_languages      |Presence in Languages Test*      |
|def_low_vision              |Has Low Vision Deficiency?    |presence_math           |Presence in Math Test*           |
|def_blind                   |Is Blind?                     |grade_natural_science   |Grade in Natural Science Test    |
|def_deaf                    |Is Deaf?                      |grade_human_science     |Grade in Human Science Test      |
|def_low_hearing             |Has Low Hearing Deficiency?   |grade_languages         |Grade in Languages Test          |
|def_blind_deaf              |Is Blind and Deaf?            |grade_math              |Grade in Math Test               |
|def_physical                |Has Physical Deficiency?      |essay_status            |Essay Status*                    |
|def_mental                  |Has Mental Deficiency?        |grade_essay             |Grade in Essay                   |

##### \*Categorical Data Value Legend
| value | presence_*             | matiral_status               | high_school_status                                  | school_type    | nationality                 | color_race   |
|-------|------------------------|------------------------------|-----------------------------------------------------|----------------|-----------------------------|--------------|
| 0     | Missed the test        | Single                       | I already finished high school                      | Uninformed     | Uninformed                  | Not declared |
| 1     | Present in the test    | Married/Lives with partner   | I am studying and will finish high school in 2018   | Public school  | Brazilian                   | White        |
| 2     | Eliminated in the test | Divorced/Unmarried/Separated | I'm studying and will finish high school after 2018 | Private school | Brazilian Naturalized       | Black        |
| 3     |                        | Widowed                      | I haven't finished and I'm not in high school       | Foreign school | Foreigner                   | Parda        |
| 4     |                        |                              |                                                     |                | Brazilian born, born abroad | Yellow       |
| 5     |                        |                              |                                                     |                |                             | Indigenous   |

| value | high_school_year_conclusion | value | high_school_year_conclusion | value | gender |
|-------|-----------------------------|-------|-----------------------------|-------|--------|
| 0     | Uninformed                  | 7     | 2011                        | M     | Male   |
| 1     | 2017                        | 8     | 2010                        | F     | Female |
| 2     | 2016                        | 9     | 2009                        |
| 3     | 2015                        | 10    | 2008                        |
| 4     | 2014                        | 11    | 2007                        |
| 5     | 2013                        | 12    | Before 2007                 |
| 6     | 2012                        |

| value | essay_status                   |
|-------|--------------------------------|
| 0     |Missed the test                 |
| 1     |Smoothly                        |
| 2     |Canceled                        |
| 3     |Copy Motivating Text            |
| 4     |In blank                        |
| 6     |Escape to the theme             |
| 7     |Non-compliance with textual type|
| 8     |Insufficient text               |
| 9     |Disconnected party              |


### Gathering Data

In order to carry out the data processing process of the ENEM dataset, I assigned zero values to the fields due to the fact that they were grades of students who missed the test, removed duplicate data by the enrollment number and renamed the columns to English, but keeping their semantic sense. Also, I kept only the columns I want to analyze. <font size="1">Although the function keeps only the columns I want to analyze, the dataset I made available did not contain any other columns. The dataset provided by INEP was very large and I reduced it to use in the project.</font>

For the Brazilian cities dataset, the columns duplicated by city and state were dropped, missing data filled in with zeros, columns renamed for better semantic sense and only columns for analysis were kept.

In [4]:
# # Comment this if you have already done the data processing process.

# # Data is processed by the functions created in the treatData.py library
# treatEnem(enem_2018_path)
# treatCities(brazil_cities_path)

Once processed, the data is placed in another folder.

In [5]:
# Processed dataset path ENEM
enem_analysis_path = os.getcwd() + '/data/analysis/enem_analysis.csv'
enem_analysis_df = pd.read_csv(enem_analysis_path, delimiter=";")

# If you want to perform a quick test, uncomment this line.
# This line takes only 1% of the dataset evenly.
enem_analysis_df = enem_analysis_df.drop(enem_analysis_df.index.difference([x for x in range(0, len(enem_analysis_df), 100)]))

print(enem_analysis_df.shape)
enem_analysis_df

(55138, 41)


,registration,city_residence_code,city_residence,state_residence_code,state_residence,age,gender,matiral_status,color_race,nationality,...,presence_natural_science,presence_human_science,presence_languages,presence_math,grade_natural_science,grade_human_science,grade_languages,grade_math,essay_status,grade_essay
0,180008202043,5300108,Brasília,53,DF,44.0,M,1.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,180008787997,2111300,São Luís,21,MA,17.0,M,0.0,3,1,...,1.0,1.0,1.0,1.0,492.2,659.1,587.2,590.5,1.0,920.0
200,180008202064,5107958,Tangará da Serra,51,MT,28.0,F,0.0,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,180008411132,4305108,Caxias do Sul,43,RS,44.0,F,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,180008196190,1100338,Nova Mamoré,11,RO,27.0,F,0.0,1,1,...,1.0,1.0,1.0,1.0,543.0,560.5,572.1,651.3,1.0,480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513300,180013592356,1302504,Manacapuru,13,AM,19.0,M,0.0,1,1,...,0.0,1.0,1.0,0.0,0.0,403.0,447.6,0.0,1.0,380.0
5513400,180013075385,1100056,Cerejeiras,11,RO,18.0,M,0.0,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5513500,180013831993,5002704,Campo Grande,50,MS,32.0,M,0.0,3,1,...,1.0,1.0,1.0,1.0,438.9,514.4,422.3,492.9,1.0,260.0
5513600,180013663323,2408003,Mossoró,24,RN,50.0,M,0.0,3,1,...,0.0,1.0,1.0,0.0,0.0,484.4,489.9,0.0,4.0,0.0


In [6]:
# Processed dataset path Brazil Cities
cities_analysis_path = os.getcwd() + '/data/analysis/cities_analysis.csv'
cities_analysis_df = pd.read_csv(cities_analysis_path, delimiter=";")

print(cities_analysis_df.shape)
cities_analysis_df

(5573, 11)


,city,state,capital,hdi_ranking,hdi,hdi_gni,hdi_life,hdi_education,longitude,latitude,altitude
0,Abadia De Goiás,GO,0,1689.0,0.708,0.687,0.830,0.622,-49.440548,-16.758812,893.60
1,Abadia Dos Dourados,MG,0,2207.0,0.690,0.693,0.839,0.563,-47.396832,-18.487565,753.12
2,Abadiânia,GO,0,2202.0,0.690,0.671,0.841,0.579,-48.718812,-16.182672,1017.55
3,Abaeté,MG,0,1994.0,0.698,0.720,0.848,0.556,-45.446191,-19.155848,644.74
4,Abaetetuba,PA,0,3530.0,0.628,0.579,0.798,0.537,-48.884404,-1.723470,10.12
...,...,...,...,...,...,...,...,...,...,...,...
5568,Xique-Xique,BA,0,4533.0,0.585,0.563,0.741,0.479,-42.725508,-10.824974,406.26
5569,Zabelê,PB,0,3639.0,0.623,0.567,0.725,0.587,-37.093552,-8.076874,646.34
5570,Zacarias,SP,0,1072.0,0.730,0.695,0.826,0.674,-50.055740,-21.050110,415.85
5571,Zé Doca,MA,0,4272.0,0.595,0.559,0.745,0.505,-45.657698,-3.275481,35.66


### Wangling Data
In order to answer the questions asked in step 1, we are going to define some functions to start working with the data to get what we want.\
<font size="1">At this point, it would be interesting to use a database to facilitate the integration of the two datasets, however, I will be using only python to browse the files for learning purposes.</font>

The `statisticalAnalysis.py` library contains functions for performing statistical analysis. Starting in an easy way, we will average the tests and analyze the highest and lowest grades:
- `avgGrade`: Returns average grade of each test.
    - Parameters:
        - *enem (pandas dataframe)*: ENEM dataframe that contains all grades.
        - *onlyPresent (boolean)*: TRUE if you only want the grades of the students who took the test. FALSE if you want all the grades.
    - Returns:
        - Dictionary with test content and their respective averages, min e max grades.

In order to answer the first question in step 1, the `gradeHDIRelation` function relates the grades to the educational HDI of each city:
- `gradeHDIRelation`: 
    - Parameters:
        - * *:
    - Returns:
        - A

## 4. Data Model
For this project, I will not be applying any artificial intelligence algorithm, but only statistical analysis. So there is nothing special about this section.

## 5. Evaluate the Results

Average student grades:

In [7]:
resAll = avgGrade(enem_analysis_df, onlyPresent=False)
resPresent = avgGrade(enem_analysis_df)

In [8]:
print('Average grades of all students:\n---------------  -------  -----  -----')
print_pretty(resAll, tabular=True)
print('\nAverage grades of students who took the test:\n---------------  ----------  -------  -----  -----')
print_pretty(resPresent, tabular=True)

Average grades of all students:
---------------  -------  -----  -----
content             mean    min    max
---------------  -------  -----  -----
natural_science  349.653      0  819
human_science    427.945      0  821
languages        396.418      0  792.7
math             379.21       0  988.4
essay            382.517      0  980

Average grades of students who took the test:
---------------  ----------  -------  -----  -----
content            presence     mean    min    max
---------------  ----------  -------  -----  -----
natural_science       39144  492.518      0  819
human_science         41595  567.281      0  821
languages             41595  525.489      0  792.7
math                  39144  534.153      0  988.4
essay                 41574  507.318      0  980


## 6. Implementation